In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv('all_data.csv')

df.columns

Index(['id', 'comment_text', 'split', 'created_date', 'publication_id',
       'parent_id', 'article_id', 'rating', 'funny', 'wow', 'sad', 'likes',
       'disagree', 'toxicity', 'severe_toxicity', 'obscene', 'sexual_explicit',
       'identity_attack', 'insult', 'threat', 'male', 'female', 'transgender',
       'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual',
       'other_sexual_orientation', 'christian', 'jewish', 'muslim', 'hindu',
       'buddhist', 'atheist', 'other_religion', 'black', 'white', 'asian',
       'latino', 'other_race_or_ethnicity', 'physical_disability',
       'intellectual_or_learning_disability', 'psychiatric_or_mental_illness',
       'other_disability', 'identity_annotator_count',
       'toxicity_annotator_count'],
      dtype='object')

In [4]:
newdf = df[['id', 'comment_text', 'split', 'publication_id',
       'parent_id', 'article_id', 'toxicity', 'severe_toxicity']]

newdf = newdf.sort_values(by='severe_toxicity', ascending=False)
newdf

,id,comment_text,split,publication_id,parent_id,article_id,toxicity,severe_toxicity
596097,5801652,Awesome! Lets cut the head off hate! Lets stab...,train,21,NaN,368010,1.000000,1.000000
207189,5997855,You are murdering feckless SCUM. Your mother ...,train,102,NaN,379957,0.983501,0.644363
601973,5883664,.\n.\nIs there really a God ?\n... I once thou...,train,53,5879007.0,373036,0.900000,0.600000
461912,6180251,DOGS----\nEAT\nSHIT\nSLEEP\nBARK\n\nNIGERS\nEA...,train,21,6179746.0,390746,0.990396,0.591236
528865,5754942,BURN THIS DOG RAPING WHITE NIGER ALIVE\nTIME f...,train,105,NaN,365110,0.973936,0.569070
...,...,...,...,...,...,...,...,...
709894,5163549,"Burnsie: I wish you all the best, but despite ...",train,54,5163479.0,328802,0.000000,0.000000
709893,5687703,Yup.,train,43,5687001.0,360936,0.000000,0.000000
709892,5121964,ANOTHER moped rider speeding AND not wearing a...,train,55,NaN,326223,0.000000,0.000000
709891,5317323,"Scheer was born in Ottawa, Ontario, 1979\n-stu...",train,54,NaN,338336,0.000000,0.000000


In [15]:
import re
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'http\S+', '', text) # Remove URLs
        text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
        text = text.lower()
        # Handle self-censored words and emojis as needed
        pass
    
    else:
        return ""
    return text

cleaned = df['comment_text'].apply(clean_text)
cleaned


0          he got his money now he lies in wait till afte...
1          mad dog will surely put the liberals in mental...
2          and trump continues his lifelong cowardice by ...
3          while arresting a man for resisting arrest\n\n...
4               tucker and paul are both total bad ass mofos
                                 ...                        
1999511    another man shamming article if white men did ...
1999512    no matter what is put in front of you regardin...
1999513    the democrat party aided and abetted by its ms...
1999514    i just dont find her a very good representatio...
1999515    you know the trump fanatics are trolling the g...
Name: comment_text, Length: 1999516, dtype: object

In [10]:
from gensim.models.doc2vec import Doc2Vec,\
    TaggedDocument
from nltk.tokenize import word_tokenize

In [18]:
# preprocess to make a big list of sentences
tagged_data = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(cleaned)]


In [20]:
import pickle

# save tagged_data
with open('tagged_data.pickle', 'wb') as handle:
    pickle.dump(tagged_data, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [21]:
# train the Doc2vec model
model = Doc2Vec(vector_size=20,
                min_count=2, epochs=10)
model.build_vocab(tagged_data)
model.train(tagged_data,
            total_examples=model.corpus_count,
            epochs=model.epochs)
 
# get the document vectors
document_vectors = [model.infer_vector(
    word_tokenize(doc)) for doc in cleaned]
 
#  print the document vectors
for i, doc in enumerate(cleaned):
    print("Document", i+1, ":", doc)
    print("Vector:", document_vectors[i])
    print()